In [2]:
library(data.table)
library(dplyr)
library(bit)
library(zoo)

## REVIEW COUNT-BASED VARIABLES

### 1. Read raw data (i.e., reviews)

In [3]:
rws=fread("../data/rws.all.csv")

In [4]:
rws %>% summarise(host.cnt=n_distinct(host_id),
                  guest.cnt=n_distinct(guest_id),
                  ls.cnt=n_distinct(listing_id))

rws %>% group_by(instant_bookable) %>% 
    summarise(ls.cnt=n_distinct(listing_id),
              host.cnt=n_distinct(host_id),
              rws.cnt=n_distinct(id),
              guest.cnt=n_distinct(guest_id))

host.cnt,guest.cnt,ls.cnt
<int>,<int>,<int>
31492,876221,39322


instant_bookable,ls.cnt,host.cnt,rws.cnt,guest.cnt
<chr>,<int>,<int>,<int>,<int>
f,25846,21842,600190,544136
t,13476,11449,382973,366686


### 2. Read facial attributes for both hosts and guests and merge those attributes with the review data

In [5]:
guest=fread("../data/guest.attribute.csv")
colnames(guest)[c(5,8)]=c("g.num_face","g.ethnicity")
guest=guest[,c("guest_id","g.num_face","g.ethnicity")]
rws.gh=left_join(rws,guest, by="guest_id")

In [6]:
host=fread("../data/host.attribute.csv")
colnames(host)[c(5,8)]=c("h.num_face","h.ethnicity")
host=host[,c("listing_id","host_id","h.num_face","h.ethnicity")]
rws.gh=left_join(rws.gh,host, by=c("listing_id","host_id"))

In [7]:
rws.gh.t=rws.gh[which(rws.gh$instant_bookable=="t"),]

host.ins=rws.gh.t[!duplicated(rws.gh.t$host_id),]
# % of host profiles with no face: 18.482%
nrow(host.ins[host.ins$h.num_face==0,])/nrow(host.ins)*100
# % of host profiles with more than one face: 11.433%
nrow(host.ins[host.ins$h.num_face>1,])/nrow(host.ins)*100

guest.ins=rws.gh.t[!duplicated(rws.gh.t$guest_id),]
# % of guest profiles with no face: 7.436%
nrow(guest.ins[guest.ins$g.num_face==0,])/nrow(guest.ins)*100
# % of guest profiles with more than one face: 21.206%
nrow(guest.ins[guest.ins$g.num_face>1,])/nrow(guest.ins)*100

[1] 18.48196

[1] 11.43331

[1] 7.436062

[1] 21.20615

### 3. Take those with only one face in the profile photo

In [8]:
rws.gh=rws.gh[rws.gh$g.ethnicity!="",]
rws.gh=rws.gh[rws.gh$h.ethnicity!="",]

In [9]:
unique(rws.gh$h.num_face)

[1] 1

In [10]:
host.ins=rws.gh[!duplicated(rws.gh$host_id),]
table(host.ins$h.ethnicity)/nrow(host.ins)*100
# ASIAN    BLACK    WHITE 
# 21.066   20.928   58.006 
guest.ins=rws.gh[!duplicated(rws.gh$guest_id),]
table(guest.ins$g.ethnicity)/nrow(guest.ins)*100
# ASIAN    BLACK    WHITE 
# 23.480   15.985   60.534 


   ASIAN    BLACK    WHITE 
21.06643 20.92760 58.00596 


   ASIAN    BLACK    WHITE 
23.48034 15.98534 60.53432 

### 4. Merge with listing vars

#### 4.1. Read listing data

In [12]:
ls=fread("../data/ls.with.rws.csv")
ls=ls[,c("listing_id","price","review_scores_rating","room_type")]
ls$price=as.numeric(gsub('[$,]', '', ls$price))

#### 4.2. Merge listing vars with the processed instant booking data

In [13]:
rws.gh=left_join(rws.gh,ls, by="listing_id")

#-Change the column name
colnames(rws.gh)[which(colnames(rws.gh)=="review_scores_rating")]="rating"

#### 4.3 Create a copy of the main data table

- Keep the main data table as we will create different versions of data, e.g., whether to include negative endorsements.

In [14]:
df=rws.gh

### 5. Create booking- / staying-level reputation and endorsement signals

#### 5.1. Create columns for # of previous reviews and race of previous reviewers for each booking/staying

In [15]:
df=df %>% group_by(listing_id) %>% arrange(date) %>%
  mutate(cum.cnt=row_number()-1,
         rws.race=lag(g.ethnicity, n=1, default = NA))

- Regarding **df\\$rws.race[which(is.na(df\\$rws.race))]**,

- Should we need to exclude these guests (who probably the first guests to hosts) in the analysis for the same-race endorsement effects? Only those with at least 6 reviews will be included in the main analysis for the effects of the same-race endorsements!

- These guests might be good data points for estimating the baseline homophily/discrimination though! See **../figs_supp/add_supp_baseline-homophily.ipynb** for additional robustness checks of our **Fig. 1**.

#### 5.2. Count the previous guest race for each booking

In [16]:
#-Create column and add counter
df$g.W.cnt=ifelse(df$rws.race=="WHITE", 1, 0)
df$g.B.cnt=ifelse(df$rws.race=="BLACK", 1, 0)
df$g.A.cnt=ifelse(df$rws.race=="ASIAN", 1, 0)

In [17]:
#-Sort reviews for each listing by date
#-Then roll sum racial compositions in the most recent 6 reviews for each booking
#-Note that the most 6 reviews are the proxy of front-page reviews
df = df %>% group_by(listing_id) %>% arrange(date) %>%
    mutate(top6.W=rollsumr(g.W.cnt==1, 6, fill=NA),
           top6.B=rollsumr(g.B.cnt==1, 6, fill=NA),
           top6.A=rollsumr(g.A.cnt==1, 6, fill=NA))

In [18]:
#-Code the first review's the most recent previous reviewer's race as 0
#-and then count all the previous reviewers' races cumulatively
df$rws.race[which(is.na(df$rws.race))]=0

df = df %>% group_by(listing_id) %>% arrange(date) %>% 
    mutate(g.W.cnt=cumsum(rws.race=="WHITE"),
           g.B.cnt=cumsum(rws.race=="BLACK"),
           g.A.cnt=cumsum(rws.race=="ASIAN"))

#### 5.3. Compute the proportions of previous reviewers' races and label them into bins in 10% intervals

In [19]:
df=as.data.table(df)

df$prop.W=df$g.W.cnt/df$cum.cnt
df$prop.B=df$g.B.cnt/df$cum.cnt
df$prop.A=df$g.A.cnt/df$cum.cnt

df$prop.W2=NA
df$prop.W2=as.character(df$prop.W2)
df=df[prop.W <0.1, prop.W2 := "0-10%"]
df=df[prop.W >=0.1 & prop.W<0.2, prop.W2 := "10-20%"]
df=df[prop.W >=0.2 & prop.W<0.3, prop.W2 := "20-30%"]
df=df[prop.W >=0.3 & prop.W<0.4, prop.W2 := "30-40%"]
df=df[prop.W >=0.4 & prop.W<0.5, prop.W2 := "40-50%"]
df=df[prop.W >=0.5 & prop.W<0.6, prop.W2 := "50-60%"]
df=df[prop.W >=0.6 & prop.W<0.7, prop.W2 := "60-70%"]
df=df[prop.W >=0.7 & prop.W<0.8, prop.W2 := "70-80%"]
df=df[prop.W >=0.8 & prop.W<0.9, prop.W2 := "80-90%"]
df=df[prop.W >=0.9,  prop.W2 := "90-100%"]
table(df$prop.W2)

df$prop.B2=NA
df$prop.B2=as.character(df$prop.B2)
df=df[prop.B <0.1, prop.B2 := "0-10%"]
df=df[prop.B >=0.1 & prop.B<0.2, prop.B2 := "10-20%"]
df=df[prop.B >=0.2 & prop.B<0.3, prop.B2 := "20-30%"]
df=df[prop.B >=0.3 & prop.B<0.4, prop.B2 := "30-40%"]
df=df[prop.B >=0.4 & prop.B<0.5, prop.B2 := "40-50%"]
df=df[prop.B >=0.5 & prop.B<0.6, prop.B2 := "50-60%"]
df=df[prop.B >=0.6 & prop.B<0.7, prop.B2 := "60-70%"]
df=df[prop.B >=0.7 & prop.B<0.8, prop.B2 := "70-80%"]
df=df[prop.B >=0.8 & prop.B<0.9, prop.B2 := "80-90%"]
df=df[prop.B >=0.9,  prop.B2 := "90-100%"]
table(df$prop.B2)

df$prop.A2=NA
df$prop.A2=as.character(df$prop.A2)
df=df[prop.A <0.1, prop.A2 := "0-10%"]
df=df[prop.A >=0.1 & prop.A<0.2, prop.A2 := "10-20%"]
df=df[prop.A >=0.2 & prop.A<0.3, prop.A2 := "20-30%"]
df=df[prop.A >=0.3 & prop.A<0.4, prop.A2 := "30-40%"]
df=df[prop.A >=0.4 & prop.A<0.5, prop.A2 := "40-50%"]
df=df[prop.A >=0.5 & prop.A<0.6, prop.A2 := "50-60%"]
df=df[prop.A >=0.6 & prop.A<0.7, prop.A2 := "60-70%"]
df=df[prop.A >=0.7 & prop.A<0.8, prop.A2 := "70-80%"]
df=df[prop.A >=0.8 & prop.A<0.9, prop.A2 := "80-90%"]
df=df[prop.A >=0.9,  prop.A2 := "90-100%"]
table(df$prop.A2)


  0-10%  10-20%  20-30%  30-40%  40-50%  50-60%  60-70%  70-80%  80-90% 90-100% 
  12597    2060    8072   16413   27825   69393   81634   52513   20278   23505 


  0-10%  10-20%  20-30%  30-40%  40-50%  50-60%  60-70%  70-80%  80-90% 90-100% 
 114002  107299   49977   19161    6982    8651    2755     853     356    4254 


  0-10%  10-20%  20-30%  30-40%  40-50%  50-60%  60-70%  70-80%  80-90% 90-100% 
  57156   78215   96400   44407   14422   13108    3966     961     354    5301 

In [20]:
df=as.data.table(df)
df

listing_id,host_id,instant_bookable,id,date,guest_id,reviewer_name,gld_id,g.num_face,g.ethnicity,⋯,g.A.cnt,top6.W,top6.B,top6.A,prop.W,prop.B,prop.A,prop.W2,prop.B2,prop.A2
<int>,<int>,<chr>,<int>,<date>,<int>,<chr>,<chr>,<int>,<chr>,⋯,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
5441,7989,f,903,2009-03-12,8512,Stephen,8512-5441-2009-03-12,1,WHITE,⋯,0,NA,NA,NA,NaN,NaN,NaN,NA,NA,NA
5441,7989,f,2179,2009-05-09,9818,Daniela,9818-5441-2009-05-09,1,ASIAN,⋯,0,NA,NA,NA,1.0000000,0.0000000,0.0000000,90-100%,0-10%,0-10%
5803,9744,f,2767,2009-05-17,16345,Jocelyne,16345-5803-2009-05-17,1,BLACK,⋯,0,NA,NA,NA,NaN,NaN,NaN,NA,NA,NA
6848,15991,t,3149,2009-05-25,18003,Ken,18003-6848-2009-05-25,1,WHITE,⋯,0,NA,NA,NA,NaN,NaN,NaN,NA,NA,NA
6848,15991,t,3287,2009-05-29,16426,Mimi,16426-6848-2009-05-29,1,WHITE,⋯,0,NA,NA,NA,1.0000000,0.0000000,0.0000000,90-100%,0-10%,0-10%
5441,7989,f,3307,2009-05-30,13223,Lizeth,13223-5441-2009-05-30,1,WHITE,⋯,1,NA,NA,NA,0.5000000,0.0000000,0.5000000,50-60%,0-10%,50-60%
5803,9744,f,3549,2009-06-03,19091,Cam,19091-5803-2009-06-03,1,WHITE,⋯,0,NA,NA,NA,0.0000000,1.0000000,0.0000000,0-10%,90-100%,0-10%
6848,15991,t,3687,2009-06-07,18432,Alice,18432-6848-2009-06-07,1,WHITE,⋯,0,NA,NA,NA,1.0000000,0.0000000,0.0000000,90-100%,0-10%,0-10%
6848,15991,t,3987,2009-06-14,8646,Mike,8646-6848-2009-06-14,1,ASIAN,⋯,0,NA,NA,NA,1.0000000,0.0000000,0.0000000,90-100%,0-10%,0-10%


### 6. Save the processed data

In [21]:
tmp = aggregate(data = df[,c("instant_bookable", "listing_id")],
                instant_bookable ~ listing_id,
                function(instant_bookable) length(unique(instant_bookable)))
unique(tmp$instant_bookable)

[1] 1

In [22]:
rws.gh.t=df[which(df$instant_bookable=="t"),]

write.csv(rws.gh.t, file="../data/df_processed-rws_instant_bookings.csv", row.names=F)